# ДЗ 1, часть 1. Метод максимального правдоподобия

Имя, Фамилия: *напишите здесь*

группа: *напишите здесь группу, где вы числитесь*

**Оценка(для проверяющего):** 0 из 10

**Дедлайн:** одновременно с частью 3

In [1]:
#all imports here please
import pickle as pkl

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#... add your own if necessary

## Задача 1. Свойства ММП оценок для модели нормального распределения (2 балла)

Рассмотрим следующую модель из независимых векторных наблюдений $X_1,..,X_n$:

$$X_i \overset{iid}{\sim} \mathcal{N}(\mu, \Sigma).$$

Постройте и исследуйте оценку для вектора матожиданий $\mu$ и ковариационной матрицы $\Sigma$ методом максимального правдоподобия.

**В помощь:** Если дифференцировать функцию $f: \mathbb{R}^{m \times n} \to \mathbb{R}$ по ячейкам входной матрицы, то получится матрица из производных, обозначаемая $\nabla_A f(A)$. Условие первого порядка для оптимизации записывается так же, как и в привычных вам случаях, нужно приравнять производные к нулю. Можете использовать [таблицу](https://en.wikipedia.org/wiki/Matrix_calculus#Scalar-by-matrix_identities) для получения оценок.

### Правдоподобие

*Запишите функцию правдоподобия и ее логарифм.*

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### ММП-оценка

*Найдите выражение для ММП-оценки и сравните решение с решением в одномерном случае.*

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

## Задача 2. Свойства ММП оценок для модели равномерного распределения (2 балла)

Рассмотрим модель выборки $X_1,..,X_n$ из наблюдений

$X_i \overset{iid}{\sim} U[0,a]$

### Правдоподобие

*Запишите функцию правдоподобия и ее логарифм.*

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### ММП-оценка параметра $a$

*Найдите выражение для ММП-оценки параметра $a$*.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

## Задача 3. Оцениваем модель охлаждения (3 балла)

[(Другой) закон Ньютона](https://en.wikipedia.org/wiki/Newton%27s_law_of_cooling)  -- простая математическая модель охлаждения(или нагревания) тела

$$
\frac{dT}{dt} = r (T_{env} - T(t)),
$$

это дифференциальное уравнение первого порядка, которое в непрерывном (то есть, в любой момент $t$) показывает, какая должна быть температура у тела, если дана начальная температура $T(0)=T_0$. Температурные единицы измерения -- Кельвины, $r$ -- это некоторая константа, зависящая от самого тела, от состава среды и т.п. . Это приближённый закон.

Вообще константа $r$ неизвестна априори и если мы хотим делать какое-то устройство, которое использует такую модель, мы должны как-то оценить $r$. Один из вариантов -- померить температуру среды (полагаем её константной), нагреть тело до какой-то температуры $T_0$, а потом оставить остывать и замерять через разные промежутки времени его температуру. По итогам таких наблюдений получится датасет
$$
T_{t_0},T_{t_1},..,T_{t_n}, ~~ t_0=0.
$$

Осталось придумать вероятностную модель. Например, мы можем записать приближённое дискретизированное по времени уравнение, используя метод Эйлера:

$$
T_{t_{i+1}} = T_{t_{i}} + r (T_{env} - T_{t_{i}}) \Delta t_i, ~~  \Delta t_i = t_{i+1} - t_i,
$$

а дальше предлагать разные варианты оценивания.

### Правдоподобие

С учётом ошибок измерений и необъяснимых факторов среды, договорившись измерять температуру через равные промежутки времени $\Delta t_i= \Delta t$, можно предположить модель, очень похожую на модель авторегрессии $AR(1)$:

$$
T_{i+1} = (1 - r\Delta t)T_{i} + r T_{env} \Delta t + \varepsilon_{i+1}, ~~ \varepsilon_{i+1} \sim_{iid} N(0,\sigma^2).
$$

1. Предполагая $T_{env}$ и $\Delta t$ данными, определите параметры модели, которые нужно оценивать и выведите функцию лог-правдоподобия.

2. Выведите оценки ММП для данной модели.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### Пишем модель

Имплементируйте класс модели, используя шаблон ниже и проведите некоторые тесты, чтобы убедиться, что он рабочий.

In [2]:
class CoolingModel:
    
    def __init__(self, r, delta, sigma2):
        self.r = r
        self.sigma2 = sigma2
        self.delta = delta
        
    def step(self,T0,Tenv):
        """Makes one step in time

        Args:
            T0 (float[]): current temperature (NTraj,)
            Tenv (float[]): current environment temperature (NTraj,)
        Returns:
            T (float[]): next temperature (NTraj,)
        """        
        raise NotImplementedError
    
    def simulate(self, T0, NSteps, Tenv):
        """Simulates NTraj trajectories of cooling process

        Args:
            T0 (float[]): initial temperature (NTraj,)
            NSteps (int): number of time steps
            Tenv (float): environment temperature
        Returns:
            trajs (float[]): simulation result (NTraj,NSteps)
        """        
        Tenv = np.ones_like(T0)*Tenv
        trajs = np.zeros([T0.shape[0],NSteps+1])
        trajs[:,0] = T0
        #YOUR CODE? use self.step
        # trajs = ....s
        return trajs       

    def fit(self, temps, Tenv):
        """Fits the model using the given data

        Args:
            temps (float[]): temperatures (N,)
            Tenv (float): environment temperature
        """        
        #write here your MLE estimate (see the task below)
        raise NotImplementedError

Попробуйте для нарисовать несколько возможных траекторий, код ниже должен показать график.

In [3]:
#params
r = 0.2 #1/sec
sigma2=0.02

#time stuff
NSteps = 1000
finalTime = 100
delta=finalTime/NSteps #sec

#number of simulations
NTrajs = 5

#temperatures
T0 = np.ones([NTrajs])*280 #K
Tenv = 300 #K

procModel = CoolingModel(r,delta,sigma2)
trajs = procModel.simulate(T0,NSteps,Tenv)

In [ ]:
ts = np.arange(0,finalTime+delta/2,delta) #+delta/2 to include the last value

f, ax = plt.subplots(figsize=(5,3))

ax.grid()
ax.set_title("Симуляции процесса охлаждения")
ax.set_xlabel("t, сек")
ax.set_ylabel("T, K")
ax.plot(ts,trajs.T)
plt.show()

ВАШ КОММЕНТАРИЙ ЗДЕСЬ

### Оценка модели по данным

Допишите метод `fit` в класс модели и оцените параметры, используя данный датасет, с помощью метода максимального правдоподобия. Данные в формате ``pkl``, загружаются как словарь.

In [7]:
with open("./coolingModel.pkl","rb") as f:
    dd = pkl.load(f)
    #YOUR CODE
    #read the data, run fit below

In [ ]:
#random init
procModel = CoolingModel(0.1,delta,0.1)
#procModel.fit(??,??)
print(f"Parameters:  r={procModel.r},sigma^2={procModel.sigma2}")

Нарисуйте несколько траекторий оценённой модели более тонкими и прозрачными линиями и сравните с вашими данными.

In [9]:
#trajs = procModel.simulate(???)

In [ ]:
ts = np.arange(0,finalTime+delta/2,delta) #+delta/2 to include the last value

f, ax = plt.subplots(figsize=(5,3))

ax.grid()
ax.set_title("Симуляции процесса охлаждения")
ax.set_xlabel("t, сек")
ax.set_ylabel("T, K")
#YOUR CODE
#any plots?

plt.show()

ВАШ КОММЕНТАРИЙ ЗДЕСЬ.

## Задача 4. Дискретная модель цен (3 балла)

### Броуновское движение

[Броуновское движение (или по-другому Винеровский процесс)](https://en.wikipedia.org/wiki/Wiener_process)  - это случайный процесс (набор случайных величин) $\{W_t\}_{t \geq 0}$ c непрерывным временем, который задётся так:

* $W_0 = 0$

* $W_t - W_s \sim \mathcal{N}(0, t-s)$, $0 \leq s \leq t$

* $W_{t_2} - W_{t_1}$ и $W_{t_4} - W_{t_3}$  независимы для $0 \leq t_1 \leq t_2 \leq t_3 \leq t_4$

Мы можем попытаться нарисовать значения процесса в любом конечном наборе точек $(t_i)_{i=0}^{N_t}$, это будут зависимые случайные величины. Удобнее всего для симуляции пользоваться последним свойством: если обозначить $\Delta t_i = t_{i+1} - t_i$, то

$$
W_{t_{i+1}} = W_{t_i} + \varepsilon_{i+1},
$$

где $W_{t_0}$ задано, а шумы $\varepsilon_{i+1} \sim N(0, \Delta t_i)$ независимы.

### Симуляция

*Реализуйте симуляцию этого дискретизированного процесса $(W_{t_i})_{i=0}^{N_t}$, используя данную сетку по времени $(t_i)_{i=0}^{N_t}$.*

In [11]:
class BrownianMotion:

    def __init__(self):
        pass

    def sample(self, ts, Ntraj):
        """Returns a discrete set of observations

        Args:
            ts (float[]): time grid (T,)
            Ntraj (int): number of trajectories to sample
        Returns
            trajs (float[]): (Ntraj,T) samples
        """
        trajs = np.zeros([Ntraj,len(ts)])
        #YOUR CODE
        #trajs = ??
        return trajs

Проверим! Нарисуйте набор семплов наблюдений $W_{t_0},..,W_{t_{N_{t}}}$ из модели Броуновского движения при $t_0=0$ и $W_{t_0} = 0$.

In [12]:
#example
Ntraj = 50
T=3
Nt = 100#number of time subdivisions
delta = T/Nt
ts = np.arange(0,T+delta/2 ,delta)# discrete points from [0,T] 

bmProc = BrownianMotion()
trajs = bmProc.sample(ts,Ntraj)

In [ ]:
f, ax = plt.subplots(figsize=(7,4))

ax.grid()
ax.set_title("Траектории Броуновского движения")
ax.set_xlabel("t")
ax.plot(ts,trajs.T)
plt.show()#if needed

## Геометрическое Броуновское движение

Геометрическое Броуновское движение (GBM) -- это модель

$$
S_t = S_0 \exp[(\mu - \frac{\sigma^2}{2})t + \sigma W_t],
$$

где параметр $\mu$ называется сносом (он связан с безрисковой процентной ставкой на рынке и выплатами по активу, например, дивидендами акции), а $\sigma$ -- волатильностью. Это один из базовых кирпичиков моделей финансовых рядов.

Пусть мы пронаблюдали набор значений $S_{t_1},..,S_{t_N}$, давайте получим оценки параметров и попробуем оценить их по данным.


### Параметры

*Определите, какие параметры модели вы будете оценивать.*

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### Правдоподобие

*Запишите функцию правдоподобия для модели данных и ее логарифм.*

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### ММП-оценки

*Найдите выражение для ММП-оценок параметров.*


ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### Пишем модель

In [14]:
class GBM:

    def __init__(self,mu,sigma):
        self.mu = mu 
        self.sigma = sigma
        self.brownMotion = BrownianMotion() 

    def sample(self, x0, ts):
        """Returns a discrete set of observations

        Args:
            x0 (float[]): set of initial conditions x_{ts[0]}  (Ntraj,)
            ts (float[]): time grid (T,)
        Returns
            trajs (float[]): (Ntraj,T) samples
        """
        Ntraj = x0.shape[0]
        #YOUR CODE
        #trajs =??
        return trajs

    def fit(self, st, delta):
        """Fits the parameters given the data (assuming the step size is constant and equal delta)

        Args:
            st (float[]): observations (T,)
            delta (flaot): time step size
        """
        #YOUR ESTIMATION HERE
        #self.mu = ??   self.sigma = ??s

Протестируем.

In [15]:
#example
Ntraj = 50
T=50
Nt = 5000#number of time subdivisions
delta = T/Nt
ts = np.arange(0,T+delta/2 ,delta)# discrete points from [0,T] 

mu=0.02
sigma=0.27
gbmProc = GBM(mu, sigma)

x0 = np.ones([Ntraj])*100
trajs = gbmProc.sample(x0,ts)

In [ ]:
f, ax = plt.subplots(figsize=(7,4))

ax.grid()
ax.set_title("Траектории GBM")
ax.set_xlabel("t")
ax.plot(ts,trajs.T)
plt.show()#if needed

### Оценка модели по данным

Допишите метод `fit` в класс модели и оцените параметры, используя данный датасет, с помощью метода максимального правдоподобия. Данные в формате ``pkl``, загружаются как словарь.

In [17]:
# load data
with open("./gbmData.pkl", "rb") as f:
    dd=pkl.load(f)
    #YOUR CODE
    #read the data

In [ ]:
#random init
gbmProc = GBM(0.01, 0.01)
#gbmProc.fit(??)

print('mu_MLE', gbmProc.mu)
print('sigma_MLE', gbmProc.sigma) 

Нарисуйте несколько траекторий оценённой модели более тонкими и прозрачными линиями и сравните с вашими данными.

In [21]:
#generating new trajectories
Ntraj = 10

#
#trajs = gbmProc.sample(??)

In [ ]:
f, ax = plt.subplots(figsize=(7,4))

ax.grid()
ax.set_title("Траектории GBM")
ax.set_xlabel("t")
#YOUR CODE
#any plots?
ax.legend(["Data","Samples"])
plt.show()#if needed

ВАШ КОММЕНТАРИЙ ЗДЕСЬ